In [5]:
import matplotlib.pyplot as pltwhich
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset


# Assignment 2
**By Team 23, consisting of:**

*Michaèl Kersten, 1653652*

*Rohan Shetty, 1829114*

*Joshua Schweikart Dumoulin, 1718088*